Table of Contents
1. Useful libraries have been imported
2. Zip files are uploaded in and read as dataframes
3. Some features are dropped (EDA on different notebook helped decide which features to drop)
4. Datasets are merged together
5. Categorical features are converted into numerical features
6. Missing values/null values are filled in
7. Data is standardized
8. PCA is performed
9. 70/15/15 split test is applied
10. Logistic Regression
11. Decision Tree
12. Random Forest
13. Artificial Neural Network

In [2]:
#Importing useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import DataLoader, TensorDataset

#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout
#from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

In [3]:
# List function (zip_file_paths) that stores the path to multiple zip files
zip_file_paths = [
    '/content/POS_CASH_balance.csv.zip',
    '/content/application_test.csv.zip',
    '/content/application_train.csv.zip',
    '/content/bureau.csv.zip',
    '/content/bureau_balance.csv.zip',
    '/content/credit_card_balance.csv.zip',
    '/content/installments_payments.csv.zip',
    '/content/previous_application.csv.zip'
]

In [4]:
# This creates a folder where all the zipfiles will be stored within a directory
zipfiles = 'zipfiles'
os.makedirs(zipfiles, exist_ok=True) #exist_ok=True states that if this code is run again and directory already exhists to ignore it

In [5]:
# This creates a continious loop to go through all 8 path files and extract them
for zip_file_path in zip_file_paths:
   with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
       zip_ref.extractall(zipfiles)
   print(f'Extracted Zip Files: {zip_file_path}')

FileNotFoundError: [Errno 2] No such file or directory: '/content/POS_CASH_balance.csv.zip'

In [ ]:
# This creates an empty dictionary so that when it loops through the zipfiles it converst them into a dataframe
dataframes = {}
for file in os.listdir(zipfiles):
    if file.endswith('.csv'):
        df_name = file.split('.')[0] # This is taking the name of the dataframe before the .
        dataframes[df_name] = pd.read_csv(os.path.join(zipfiles, file)) # This allows for the code to read this as the name before the .
        print(f'Read {file} into DataFrame: {df_name}')

In [ ]:
# Renaming the dataframes into new variable names
credit_card_balance = dataframes['credit_card_balance']
previous_application = dataframes['previous_application']
bureau_balance = dataframes['bureau_balance']
POS_CASH_balance = dataframes['POS_CASH_balance']
application_train = dataframes['application_train']
application_test = dataframes['application_test']
installments_payments = dataframes['installments_payments']
bureau = dataframes['bureau']

In [ ]:
# EDA was done in a different notebook when decided which features to drop
# New variable names are created for each dataset without the dropped variables
credit1 = credit_card_balance.drop(columns=[
    'SK_ID_PREV', 'AMT_PAYMENT_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
    'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_DRAWINGS_ATM_CURRENT',
    'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT'
])

bureau1 = bureau.drop(columns=[
    'CREDIT_DAY_OVERDUE', 'CNT_CREDIT_PROLONG'
])

install1 = installments_payments.drop(columns=[
    'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'NUM_INSTALMENT_VERSION',
    'NUM_INSTALMENT_NUMBER'
])

POS1 = POS_CASH_balance.drop(columns=[
    'CNT_INSTALMENT_FUTURE'
])

pre_app1 = previous_application.drop(columns=[
    'DAYS_LAST_DUE', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_INTEREST_PRIMARY',
    'AMT_GOODS_PRICE', 'AMT_APPLICATION'
])

app_train1 = application_train.drop(columns=[
    'REGION_POPULATION_RELATIVE', 'FLAG_MOBIL', 'FLAG_DOCUMENT_2',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_10',
    'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'DEF_60_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE', 'LIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
    'ELEVATORS_MEDI', 'APARTMENTS_MEDI', 'LIVINGAREA_MODE',
    'LIVINGAPARTMENTS_MODE', 'ELEVATORS_MODE', 'APARTMENTS_MODE',
    'LIVINGAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG',
    'APARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
    'LANDAREA_MEDI', 'FLOORSMIN_MEDI', 'FLOORSMAX_MEDI',
    'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ENTRANCES_MEDI',
    'FLOORSMIN_MODE', 'LANDAREA_MODE', 'FLOORSMAX_MODE',
    'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
    'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ENTRANCES_MODE',
    'LIVE_CITY_NOT_WORK_CITY', 'LIVE_REGION_NOT_WORK_REGION',
    'REGION_RATING_CLIENT', 'CNT_CHILDREN', 'AMT_GOODS_PRICE',
    'NONLIVINGAPARTMENTS_MODE', 'FLAG_DOCUMENT_15','NONLIVINGAPARTMENTS_AVG',
    'FLAG_CONT_MOBILE', 'DAYS_ID_PUBLISH', 'BASEMENTAREA_AVG'
])

app_test1 = application_test.drop(columns=[
    'REGION_POPULATION_RELATIVE', 'FLAG_MOBIL', 'FLAG_DOCUMENT_2',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_10',
    'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'DEF_60_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE', 'LIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
    'ELEVATORS_MEDI', 'APARTMENTS_MEDI', 'LIVINGAREA_MODE',
    'LIVINGAPARTMENTS_MODE', 'ELEVATORS_MODE', 'APARTMENTS_MODE',
    'LIVINGAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG',
    'APARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
    'LANDAREA_MEDI', 'FLOORSMIN_MEDI', 'FLOORSMAX_MEDI',
    'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ENTRANCES_MEDI',
    'FLOORSMIN_MODE', 'LANDAREA_MODE', 'FLOORSMAX_MODE',
    'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
    'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ENTRANCES_MODE',
    'LIVE_CITY_NOT_WORK_CITY', 'LIVE_REGION_NOT_WORK_REGION',
    'REGION_RATING_CLIENT', 'CNT_CHILDREN', 'AMT_GOODS_PRICE',
    'NONLIVINGAPARTMENTS_MODE', 'FLAG_DOCUMENT_15','NONLIVINGAPARTMENTS_AVG',
    'FLAG_CONT_MOBILE', 'DAYS_ID_PUBLISH', 'BASEMENTAREA_AVG'
])

In [ ]:
app_train1.shape

In [ ]:
app_test1.shape

In [ ]:
# This combines the app_train1 and app_test1 dataframes into one variable dataframe
combined_df = pd.concat([app_train1, app_test1], ignore_index=True, sort=False)
# ignore_index=True helps avoid duplicate indexes
# sort=False keeps the columns in order

In [ ]:
combined_df.shape

In [ ]:
bureau1.columns

In [ ]:
bureau_balance.columns

In [ ]:
bureau_balance.shape

In [ ]:
bureau1.shape

In [ ]:
# groups the dataframe bureau_balance by the feature SK_ID_BUREAU (agg is for summary)
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
    'MONTHS_BALANCE': 'count',
    'STATUS': lambda x: (x == '0').sum()
}).reset_index()

In [ ]:
# groups the dataframe bureau by the feature SK_ID_CURR (agg is for summary)
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': 'sum',
    'AMT_CREDIT_SUM_DEBT': 'sum',
    'AMT_CREDIT_SUM_LIMIT': 'sum',
    'CREDIT_ACTIVE': lambda x: (x == 'Active').sum(),
    'AMT_CREDIT_MAX_OVERDUE': 'max',
}).reset_index()

In [ ]:
# merged bureau_agg and bb_agg into the dataframe merged_bureau
merged_bureau = pd.merge(bureau_agg, bb_agg, left_on='SK_ID_CURR', right_on='SK_ID_BUREAU', how='left')

In [ ]:
merged_bureau.shape

In [ ]:
# merged combined_df and merged_bureau into a new dataframe named final_df on SK_ID_CURR
final_df = pd.merge(combined_df, merged_bureau, on='SK_ID_CURR', how='left')

In [ ]:
final_df.shape

In [ ]:
credit1.columns

In [ ]:
POS1.columns

In [ ]:
pre_app1.columns

In [ ]:
install1.columns

In [ ]:
# groups the dataframe credit1 by the SK_ID_CURR (agg is for summary)
credit1_agg = credit1.groupby('SK_ID_CURR').agg({
    'MONTHS_BALANCE': 'count',
    'AMT_BALANCE': 'sum',
    'AMT_CREDIT_LIMIT_ACTUAL': 'sum',
    'AMT_DRAWINGS_CURRENT': 'sum',
    'AMT_DRAWINGS_OTHER_CURRENT': 'sum',
    'AMT_DRAWINGS_POS_CURRENT': 'sum',
    'AMT_PAYMENT_TOTAL_CURRENT': 'sum',
    'CNT_DRAWINGS_CURRENT': 'sum',
    'NAME_CONTRACT_STATUS': lambda x: x.value_counts().get('Active', 0)
}).reset_index()

In [ ]:
# groups the dataframe POS1 by the SK_ID_CURR (agg is for summary)
POS1_agg = POS1.groupby('SK_ID_CURR').agg({
    'MONTHS_BALANCE': 'count',
    'CNT_INSTALMENT': 'sum',
    'NAME_CONTRACT_STATUS': lambda x: x.value_counts().get('Active', 0)
}).reset_index()

In [ ]:
# groups the dataframe pre_app1 by the SK_ID_CURR (agg is for summary)
pre_app1_agg = pre_app1.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'AMT_ANNUITY': 'sum',
    'AMT_CREDIT': 'sum',
    'AMT_DOWN_PAYMENT': 'sum',
    'NAME_CONTRACT_TYPE': lambda x: x.value_counts().idxmax(),
}).reset_index()

In [ ]:
# groups the dataframe install1 by the SK_ID_CURR (agg is for summary)
install1_agg = install1.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'AMT_PAYMENT': 'sum'
}).reset_index()

In [ ]:
# merging credit1_agg to the overall final dataset (the apps and bureau datasets)
final_df = final_df.merge(credit1_agg, on='SK_ID_CURR', how='left')
final_df.shape

In [ ]:
# merging POS1_agg to the overall final dataset
final_df = final_df.merge(POS1_agg, on='SK_ID_CURR', how='left')
final_df.shape

In [ ]:
# merging pre_app1_agg to the overall final dataset
final_df = final_df.merge(pre_app1_agg, on='SK_ID_CURR', how='left')
final_df.shape

In [ ]:
# merging install1_agg to the overall final dataset
final_df = final_df.merge(install1_agg, on='SK_ID_CURR', how='left')
final_df.shape

In [ ]:
# checks for mixed columns
mixed_type_cols_final = []
for col in final_df.columns: #Checks all columns in a loop
    if final_df[col].dtype == 'object': # sees if it is an object
        if final_df[col].apply(type).nunique() > 1: # checks to see if there is more than one datatype in a column
            mixed_type_cols_final.append(col) # adds all the found columns in a list

print("Columns with mixed types in final_df:", mixed_type_cols_final) # prints the list of mixed columns

In [ ]:
# converting the mixed columsn into a numerical datatype
for col in mixed_type_cols_final:
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

In [ ]:
# this is doing one hot encoding- converting the categorical features
final_df = pd.get_dummies(final_df, drop_first=True)

In [ ]:
final_df.shape

In [ ]:
# filling in missing data with 0
final_df.fillna(0, inplace=True)

In [ ]:
thisis =

In [ ]:
target_variable = final_df['TARGET']

In [ ]:
print(target_variable)

In [ ]:
variances = final_df.var()
sorted_variances = variances.sort_values(ascending=False)
top_20_features = sorted_variances.head(20)
print(top_20_features)

In [ ]:
top_20_feature_names = top_20_features.index.tolist()
top_20_feature_names.append(target_variable)  # Add the 'TARGET' column name to the list
top_20_df = final_df[top_20_feature_names]
top_20_df.head()

In [ ]:
# Define the target column name as a string
target_variable = 'TARGET'  # This is the column name, not the Series itself

# Get the list of top 20 feature names from the variance calculation
top_20_feature_names = top_20_features.index.tolist()

# Add the target column name ('TARGET') to the list of top 20 features
top_20_feature_names.append(target_variable)

# Create a new DataFrame with the top 20 features and the target column
top_20_df = final_df[top_20_feature_names]

# Display the first few rows of the DataFrame
top_20_df.head()


In [ ]:
from google.colab import files
top_20_df.to_csv('/content/20df.csv', index=False)
top_20_df.head()
os.path.exists('/content/20df.csv')
files.download('/content/20df.csv')

In [ ]:
sample = top_20_df.sample(n=1000, random_state=42)

In [ ]:
sample.shape

In [ ]:
sample.to_csv('/content/sample21.csv', index=False)
os.path.exists('/content/sample21.csv')
files.download('/content/sample21.csv')

In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(
    n_samples=12,         # 12 rows of data
    n_features=173,       # 173 features
    n_informative=3,      # 3 informative features
    n_redundant=0,        # No redundant features
    n_repeated=0,         # No repeated features
    n_classes=2,          # Binary classification (0 and 1)
    random_state=42       # Ensures reproducibility
)


In [ ]:
feature_columns = [f'Feature_{i+1}' for i in range(X.shape[1])]

# Convert to DataFrame
sample_df = pd.DataFrame(X, columns=feature_columns)
sample_df['Target'] = y

sample_df.head(12)

In [ ]:
# Identify the target column and seperating it before standardization
X = final_df.drop(columns=['TARGET'])
y = final_df['TARGET']

In [ ]:
X.head()

In [ ]:
# standardizing all the columns but the TARGET column
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
print(X_scaled[:5])

In [ ]:
final_df.columns

In [ ]:
# applying PCA for dimentionality reduction
pca = PCA()
pca.fit(X_scaled)

In [ ]:
# applying transformation
pca_transformed = pca.transform(X_scaled)

In [ ]:
# creating a new dataframe for the transformed pca features
pca_df = pd.DataFrame(data=pca_transformed, columns=[f'Principal Component {i+1}' for i in range(pca_transformed.shape[1])])

In [ ]:
pca_components_df = pd.DataFrame(pca.components_, columns=X.columns, index=[f'Principal Component {i+1}' for i in range(pca_transformed.shape[1])])
pca_components_df

In [ ]:
pca_df.head()

In [ ]:
pca_df.shape

In [ ]:
# reestablishing the X and y
X = pca_df
y = final_df['TARGET']

In [ ]:
# Doing a 70/15/15 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

In [ ]:
y_test_pred = log_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

In [ ]:
y_val_pred = log_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

In [ ]:
scores = cross_val_score(log_model, X, y, cv=5)
print("Cross-Validation Scores:", scores)

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

In [ ]:
y_test_pred = dt_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy (Decision Tree):", test_accuracy)

In [ ]:
y_val_pred = dt_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy (Decision Tree):", val_accuracy)

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy (Random Forest):", test_accuracy)

In [ ]:
y_val_pred = rf_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy (Random Forest):", val_accuracy)

In [ ]:
X_val.head()

In [ ]:
# Define the target column name as a string
target_variable = 'TARGET'  # This is the column name, not the Series itself

target_column = final_df[target_variable]
X_val_with_target = X_val.copy()  # Make a copy of X_val to avoid modifying it in place
X_val_with_target[target_variable] = target_column
X_val_with_target.head()


In [ ]:
from google.colab import files
X_val_with_target.to_csv('/content/validation.csv', index=False)
X_val_with_target.head()
os.path.exists('/content/validation.csv')
files.download('/content/validation.csv')

In [ ]:
nn_model = MLPClassifier(random_state=42, max_iter=300)
nn_model.fit(X_train, y_train)

In [ ]:
y_test_pred = nn_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy (Neural Network):", test_accuracy)

In [ ]:
y_val_pred = nn_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy (Neural Network):", val_accuracy)

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))  # Input layer
ann_model.add(Dropout(0.2))

In [ ]:
ann_model.add(Dense(units=64, activation='relu'))
ann_model.add(Dropout(0.2))

In [ ]:
ann_model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
ann_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
ann_model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
test_loss, test_accuracy = ann_model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

In [ ]:
y_val_pred = ann_model.predict(X_val)
y_val_pred = (y_val_pred > 0.5)  # Convert probabilities to binary (0 or 1)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix for Random Forest Predictions')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Get classification report
report = classification_report(y_test, y_test_pred, target_names=['Negative', 'Positive'])
print(report)

In [ ]:
prediction_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_test_pred
})

# Plot the actual vs predicted values
plt.figure(figsize=(8, 6))
sns.countplot(x='Actual', hue='Predicted', data=prediction_df)
plt.title('Actual vs Predicted Target for Random Forest')
plt.xlabel('Actual Target')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, color='blue', alpha=0.5)
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Line where Actual = Predicted
plt.title('Predicted vs Actual Values')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid(True)
plt.show()

In [ ]:
prediction_df.head()

In [ ]:
X_test.head()

In [ ]:
final_df['TARGET'].head()

In [ ]:
prediction_df.tail()

In [ ]:
X_test.tail()

In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(
    n_samples=12,         # 12 rows of data
    n_features=173,         # 3 features
    n_informative=3,      # All 3 features are informative
    n_redundant=0,        # No redundant features
    n_repeated=0,         # No repeated features
    n_classes=2,          # Binary classification (0 and 1)
    random_state=42       # Ensures reproducibility
)


In [ ]:
feature_columns = [f'Feature_{i+1}' for i in range(X.shape[1])]
sample_df = pd.DataFrame(X, columns=feature_columns)
sample_df['Target'] = y

sample_df.head(12)

In [ ]:
unknown_data = final_df.sample(n=10, random_state=42)
unknown_data.to_csv('new_data.csv', index=False)
unknown_data.head(10)

In [ ]:
unknown_data.shape

In [ ]:
unknown_data.fillna(0, inplace=True)
X_unknown = unknown_data.drop(columns=['TARGET'], errors='ignore')
X_unknown_scaled = scaler.transform(X_unknown)

In [ ]:
X_unknown_pca = pca.transform(X_unknown_scaled)
X_unknown_pca_df = pd.DataFrame(X_unknown_pca, columns=[f'Principal Component {i+1}' for i in range(X_unknown_pca.shape[1])])
y_unknown_pred = rf_model.predict(X_unknown_pca_df)

In [ ]:
unknown_data['Predicted_Target'] = y_unknown_pred

In [ ]:
cols = ['SK_ID_CURR', 'TARGET', 'Predicted_Target'] + [col for col in unknown_data.columns if col not in ['SK_ID_CURR', 'TARGET', 'Predicted_Target']]
unknown_data = unknown_data[cols]
unknown_data.head(10)

In [ ]:
sample_data = final_df.sample(n=100, random_state=42)
sample_data.to_csv('/content/sample_data.csv', index=False)
sample_data.head()

In [ ]:
os.path.exists('/content/sample_data.csv')

In [ ]:
files.download('/content/sample_data.csv')

In [ ]:
sample_data.shape

In [ ]:
print(unknown_data.columns)

In [ ]:
y_true = unknown_data['TARGET']
y_pred = unknown_data['Predicted_Target']

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix for Predicted Unknown Data')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
print(final_df.columns)

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=final_df, x='TARGET')
plt.title('Bar Chart of Target Values in Original Dataset')
plt.xlabel('TARGET')
plt.ylabel('Count')
plt.xticks(ticks=[0, 1], labels=['Non-Fraudulent (0.0)', 'Fraudulent (1.0)'], rotation=0)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=unknown_data, x='TARGET')
plt.title('Bar Chart of Target Values in Unknown Dataset')
plt.xlabel('TARGET')
plt.ylabel('Count')
plt.xticks(ticks=[0, 1], labels=['Non-Fraudulent (0.0)', 'Fraudulent (1.0)'], rotation=0)
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Optionally show all rows (be cautious with large datasets)

# Access the 4th row (index 3)
row_4 = unknown_data.iloc[3]
print(row_4)

In [ ]:
import joblib
joblib.dump(log_model, 'logistic_regression_model.pkl')
joblib.dump(dt_model, 'decision_tree_model.pkl')
joblib.dump(rf_model, 'random_forest_model.pkl')

In [ ]:
ann_model.save('ann_model.h5')

In [ ]:
from google.colab import files

files.download('logistic_regression_model.pkl')
files.download('decision_tree_model.pkl')
files.download('random_forest_model.pkl')

In [ ]:
files.download('ann_model.h5')

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import joblib
import pandas as pd
from tensorflow.keras.models import load_model

In [ ]:
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(pca, 'pca.pkl')

In [ ]:
files.download('scaler.pkl')
files.download('pca.pkl')

In [ ]:
log_model = joblib.load('logistic_regression_model.pkl')
dt_model = joblib.load('decision_tree_model.pkl')
rf_model = joblib.load('random_forest_model.pkl')
ann_model = load_model('ann_model.h5')

# Load scaler and PCA (assuming they were saved earlier)
scaler = joblib.load('scaler.pkl')
pca = joblib.load('pca.pkl')

In [ ]:
def data_source():
    # Show information about data and allow file upload
    st.title('Data Source')
    st.write("This section loads the raw data from CSV files.")

    # You can use your original code for extracting data
    # Assuming `dataframes` dictionary contains the loaded data
    zipfiles = 'zipfiles'
    dataframes = {}
    for file in os.listdir(zipfiles):
        if file.endswith('.csv'):
            df_name = file.split('.')[0]
            dataframes[df_name] = pd.read_csv(os.path.join(zipfiles, file))

    # Show a sample of the data
    if dataframes:
        for df_name, df in dataframes.items():
            st.write(f"### {df_name} Data")
            st.dataframe(df.head())


In [ ]:

def eda():
    st.title('Exploratory Data Analysis (EDA)')

    # Assuming that these datasets are loaded already (adjust paths or variables if needed)
    # Apply the feature drops for each dataset
    credit1 = credit_card_balance.drop(columns=[
       'SK_ID_PREV', 'AMT_PAYMENT_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_DRAWINGS_ATM_CURRENT',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT'
    ])

    bureau1 = bureau.drop(columns=[
       'CREDIT_DAY_OVERDUE', 'CNT_CREDIT_PROLONG'
    ])

    install1 = installments_payments.drop(columns=[
       'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER'
    ])

    POS1 = POS_CASH_balance.drop(columns=[
       'CNT_INSTALMENT_FUTURE'
    ])

    pre_app1 = previous_application.drop(columns=[
       'DAYS_LAST_DUE', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_INTEREST_PRIMARY',
       'AMT_GOODS_PRICE', 'AMT_APPLICATION'
    ])

    app_train1 = application_train.drop(columns=[
       'REGION_POPULATION_RELATIVE', 'FLAG_MOBIL', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_10',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_19',
       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'LIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
       'ELEVATORS_MEDI', 'APARTMENTS_MEDI', 'LIVINGAREA_MODE',
       'LIVINGAPARTMENTS_MODE', 'ELEVATORS_MODE', 'APARTMENTS_MODE',
       'LIVINGAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG',
       'APARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
       'LANDAREA_MEDI', 'FLOORSMIN_MEDI', 'FLOORSMAX_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ENTRANCES_MEDI',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'FLOORSMAX_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ENTRANCES_MODE',
       'LIVE_CITY_NOT_WORK_CITY', 'LIVE_REGION_NOT_WORK_REGION',
       'REGION_RATING_CLIENT', 'CNT_CHILDREN', 'AMT_GOODS_PRICE',
       'NONLIVINGAPARTMENTS_MODE', 'FLAG_DOCUMENT_15', 'NONLIVINGAPARTMENTS_AVG',
       'FLAG_CONT_MOBILE', 'DAYS_ID_PUBLISH', 'BASEMENTAREA_AVG'
    ])

    app_test1 = application_test.drop(columns=[
       'REGION_POPULATION_RELATIVE', 'FLAG_MOBIL', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_10',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_19',
       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'LIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
       'ELEVATORS_MEDI', 'APARTMENTS_MEDI', 'LIVINGAREA_MODE',
       'LIVINGAPARTMENTS_MODE', 'ELEVATORS_MODE', 'APARTMENTS_MODE',
       'LIVINGAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG',
       'APARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
       'LANDAREA_MEDI', 'FLOORSMIN_MEDI', 'FLOORSMAX_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ENTRANCES_MEDI',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'FLOORSMAX_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ENTRANCES_MODE',
       'LIVE_CITY_NOT_WORK_CITY', 'LIVE_REGION_NOT_WORK_REGION',
       'REGION_RATING_CLIENT', 'CNT_CHILDREN', 'AMT_GOODS_PRICE',
       'NONLIVINGAPARTMENTS_MODE', 'FLAG_DOCUMENT_15','NONLIVINGAPARTMENTS_AVG',
       'FLAG_CONT_MOBILE', 'DAYS_ID_PUBLISH', 'BASEMENTAREA_AVG'
    ])

    # Step 1: Combine train and test data
    combined_df = pd.concat([app_train1, app_test1], ignore_index=True, sort=False)

    # Step 2: Aggregations for different datasets (Bureau, Credit, POS, Pre-app, Install)
    bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
       'MONTHS_BALANCE': 'count',
       'STATUS': lambda x: (x == '0').sum()
    }).reset_index()

    bureau_agg = bureau1.groupby('SK_ID_CURR').agg({
       'AMT_CREDIT_SUM': 'sum',
       'AMT_CREDIT_SUM_DEBT': 'sum',
       'AMT_CREDIT_SUM_LIMIT': 'sum',
       'CREDIT_ACTIVE': lambda x: (x == 'Active').sum(),
       'AMT_CREDIT_MAX_OVERDUE': 'max',
    }).reset_index()

    # Merging bureau datasets (bureau balance with bureau)
    merged_bureau = pd.merge(bureau_agg, bb_agg, left_on='SK_ID_CURR', right_on='SK_ID_BUREAU', how='left')

    # Step 3: Merge final_df with the merged bureau data
    final_df = pd.merge(combined_df, merged_bureau, on='SK_ID_CURR', how='left')

    # Step 4: Perform aggregations on other datasets (credit1, POS1, pre_app1, install1)
    credit1_agg = credit1.groupby('SK_ID_CURR').agg({
       'MONTHS_BALANCE': 'count',
       'AMT_BALANCE': 'sum',
       'AMT_CREDIT_LIMIT_ACTUAL': 'sum',
       'AMT_DRAWINGS_CURRENT': 'sum',
       'AMT_DRAWINGS_OTHER_CURRENT': 'sum',
       'AMT_DRAWINGS_POS_CURRENT': 'sum',
       'AMT_PAYMENT_TOTAL_CURRENT': 'sum',
       'CNT_DRAWINGS_CURRENT': 'sum',
       'NAME_CONTRACT_STATUS': lambda x: x.value_counts().get('Active', 0)
    }).reset_index()

    POS1_agg = POS1.groupby('SK_ID_CURR').agg({
       'MONTHS_BALANCE': 'count',
       'CNT_INSTALMENT': 'sum',
       'NAME_CONTRACT_STATUS': lambda x: x.value_counts().get('Active', 0)
    }).reset_index()

    pre_app1_agg = pre_app1.groupby('SK_ID_CURR').agg({
       'SK_ID_PREV': 'count',
       'AMT_ANNUITY': 'sum',
       'AMT_CREDIT': 'sum',
       'AMT_DOWN_PAYMENT': 'sum',
       'NAME_CONTRACT_TYPE': lambda x: x.value_counts().idxmax(),
    }).reset_index()

    install1_agg = install1.groupby('SK_ID_CURR').agg({
       'SK_ID_PREV': 'count',
       'AMT_PAYMENT': 'sum'
    }).reset_index()

    # Merging aggregated dataframes into final_df
    final_df = final_df.merge(credit1_agg, on='SK_ID_CURR', how='left')
    final_df = final_df.merge(POS1_agg, on='SK_ID_CURR', how='left')
    final_df = final_df.merge(pre_app1_agg, on='SK_ID_CURR', how='left')
    final_df = final_df.merge(install1_agg, on='SK_ID_CURR', how='left')

    # Step 5: Handle Mixed-Type Columns
    mixed_type_cols_final = []
    for col in final_df.columns:
        if final_df[col].dtype == 'object':
            if final_df[col].apply(type).nunique() > 1:
                mixed_type_cols_final.append(col)

    # Print out mixed columns
    st.write("Columns with mixed types in final_df:", mixed_type_cols_final)

    # Convert mixed-type columns to numeric
    for col in mixed_type_cols_final:
        final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

    # Step 6: Handle Categorical Features and Get Dummies
    final_df = pd.get_dummies(final_df, drop_first=True)

    # Step 7: Fill missing values with 0
    final_df.fillna(0, inplace=True)

    # Show the resulting dataframe after all EDA steps
    st.write("Data after EDA (merged, cleaned, and processed):")
    st.dataframe(final_df.head())

In [ ]:
def training():
    st.title('Model Training')

    # Split the data
    X = final_df.drop(columns=['TARGET'])
    y = final_df['TARGET']

    X_scaled = scaler.fit_transform(X)
    pca_transformed = pca.transform(X_scaled)

    # Re-define X using PCA-transformed data
    X_train, X_temp, y_train, y_temp = train_test_split(pca_transformed, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Train and evaluate models
    models = {
        "Logistic Regression": log_model,
        "Decision Tree": dt_model,
        "Random Forest": rf_model,
        "ANN": ann_model
    }

    for model_name, model in models.items():
        st.write(f"### {model_name}")

        if model_name == 'ANN':
            ann_model.fit(X_train, y_train, epochs=50, batch_size=32)
            test_loss, test_accuracy = ann_model.evaluate(X_test, y_test)
            st.write(f"Test Accuracy: {test_accuracy}")
        else:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            st.write(f"Test Accuracy: {test_accuracy}")


In [ ]:
def prediction():
    st.title('Prediction')

    # Load new data
    st.write("Upload new data to predict outcomes.")
    uploaded_file = st.file_uploader("Choose a file")

    if uploaded_file is not None:
        new_data = pd.read_csv(uploaded_file)
        new_data.fillna(0, inplace=True)

        # Apply the same preprocessing pipeline (scaling + PCA)
        X_new = new_data.drop(columns=['TARGET'], errors='ignore')
        X_new_scaled = scaler.transform(X_new)
        X_new_pca = pca.transform(X_new_scaled)

        # Choose a model for prediction (e.g., Random Forest)
        y_new_pred = rf_model.predict(X_new_pca)
        new_data['Predicted_Target'] = y_new_pred

        st.write("Predicted Data:")
        st.dataframe(new_data)


In [ ]:
# Sidebar navigation
st.sidebar.header("Navigation")

# Buttons to navigate between steps
if st.sidebar.button("Data Source"):
    st.session_state['current_step'] = 'Data Source'

if st.sidebar.button("EDA"):
    st.session_state['current_step'] = 'EDA'

if st.sidebar.button("Training"):
    st.session_state['current_step'] = 'Training'

if st.sidebar.button("Prediction"):
    st.session_state['current_step'] = 'Prediction'

# Render the appropriate step based on user choice
if st.session_state['current_step'] == 'Data Source':
    data_source()
elif st.session_state['current_step'] == 'EDA':
    eda()
elif st.session_state['current_step'] == 'Training':
    training()
elif st.session_state['current_step'] == 'Prediction':
    prediction()
